In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display

load_dotenv(override=True)

# Simple call

In [ ]:
messages = [{"role":"user", "content":"Give me 1 sentence differentiating between meiosis and mitosis."}]

# OpenAI
client = OpenAI()
response = client.chat.completions.create(
    messages=messages,
    model='gpt-4.1-nano'
)
result = (f"From gpt-4.1-nano: {response.choices[0].message.content}")
display(Markdown(result))

# Other
client = OpenAI(api_key=os.getenv('DEEPSEEK_API_KEY'), base_url="https://api.deepseek.com/v1")
response = client.chat.completions.create(
    messages=messages,
    model='deepseek-chat'
)
result = f"From deepseek-chat: {response.choices[0].message.content}"
display(Markdown(result))

# Agent

In [ ]:
from agents import Agent, Runner
from agents import AsyncOpenAI, OpenAIResponsesModel, trace
import os

In [ ]:
instruction = "You generate question about science that can be solved by college graduates."
agent = Agent(name="Science Question Agent", instructions=instruction, model='gpt-4.1-nano')
science_question_tool = agent.as_tool(tool_name="question_maker_science", tool_description="Ask a science question.")

In [ ]:
instruction = "You generate question about art history that can be solved by college graduates."
agent = Agent(name='Art Question Agent', instructions=instruction, model='gpt-4.1-nano')
art_question_tool = agent.as_tool(tool_name='question_maker_art', tool_description="Ask an art question.")

In [ ]:
instruction = "You provide answers to science and art history questions. Your knowledge is a college graduate level."
deepseek_client = AsyncOpenAI(api_key=os.getenv('DEEPSEEK_API_CALL'), base_url='https://api.deepseek.com/v1')
deepseek_model = OpenAIResponsesModel(model='deepseek-chat', openai_client=deepseek_client)
agent = Agent(name='Answer Provider Agent', instructions=instruction, model=deepseek_model)
answer_tool = agent.as_tool(tool_name='solution_provider', tool_description="Give an answer.")

In [ ]:
lecturer_instructions = """
You give a question and answer. Your goal is to select

Follow these steps carefully:
1. Generate questions: Use question_maker_art and qeustion_maker_science tools to generate 2 different questions.
2. Evaluate: Provide answers to the 2 questions by using solution_provider tool.
3. Report: Publish the questions and answers.

"""
lecturer = Agent(
    name="Lecturer",
    instructions=lecturer_instructions,
    tools = [science_question_tool, art_question_tool, answer_tool],
    model='gpt-4.1-nano'
)

message = "Give me a question and answer so that I can learn something new today."

with trace('QandA2'):
    result = await Runner.run(lecturer, message)

    print(result.final_output)

# Agent (Async)